# National Health and Nutrition Examination Survey (NHANES)
[https://wwwn.cdc.gov/nchs/nhanes/](https://wwwn.cdc.gov/nchs/nhanes/)

Este laboratório usará dados extraídos do NHANES.

## NHANES datasets from 2013-2014

Foram extraídos dados do [NHANES 2013-2014](https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2013).

Os dados estão disponíveis em arquivos CSV no diretório `/data/nhanes`. Lá está disponível a documentação do esquema dos arquivos.

# Tarefa 1 - Esquema Relacional

Analise os arquivos CSV que estão no diretório `/data/nhanes` e escreva um esquema relacional referente às tabelas, suas chaves primárias e estrangeiras (não é SQL ainda). Coloque a chave primária com dois underlines antes e dois depois:

\__teste\__

demographic_person(__id__,gender,age)  
medication_drug(__code__,name)  
medication_use(__person_id__,__drug_code__,days_use)
- CHE -> person_id from demographic_person id
- CHE -> drug_code from medication_drug code

## Criação das Tabelas em SQL e Importação de Dados

Considere o seguinte exemplo em que é criada uma tabela em SQL e importado dados do arquivo CSV `demographic-person.csv`.

In [1]:
%defaultDatasource jdbc:h2:mem:db

In [14]:
DROP TABLE IF EXISTS Pessoa;
DROP TABLE IF EXISTS Drug;
DROP TABLE IF EXISTS Use;

In [15]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('https://raw.githubusercontent.com/MatheusASCandido/labsMC536-BancoDeDados/main/lab03/data/demographic-person.csv');

In [16]:
SELECT * FROM Pessoa;

# Tarefa 2 - Criação das Tabelas em SQL e Importação de Dados

Escreva instruções SQL para montar as duas tabelas restante, conforme definido no esquema relacional da **Tarefa 1**.

**Importante:** não defina chave primária para a tabela de uso de medicamento.

Durante a criação das tabelas, realize a importação dos dados do arquivo CSV nas tabelas criadas, conforme exemplo anterior.

In [17]:
CREATE TABLE Drug (
  Code VARCHAR(6) NOT NULL,
  Name VARCHAR NOT NULL,
  PRIMARY KEY(Code),
) AS SELECT
    code,
    name
FROM CSVREAD('https://raw.githubusercontent.com/MatheusASCandido/labsMC536-BancoDeDados/main/lab03/data/medications-drug.csv');

In [18]:
SELECT * FROM Drug;

In [19]:
CREATE TABLE Use (
  person_id VARCHAR(5) NOT NULL,
  drug_code VARCHAR(6) NOT NULL,
  days_use INTEGER,
  FOREIGN KEY(person_id)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(drug_code)
    REFERENCES Drug(Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('https://raw.githubusercontent.com/MatheusASCandido/labsMC536-BancoDeDados/main/lab03/data/medications-use.csv');

In [20]:
SELECT *FROM Use 

# Tarefa 3 - Consultas Simples

Execute as consultas a seguir.

a) Liste pessoas com idade acima de 60

In [21]:
SELECT * FROM Pessoa WHERE Age>=60;

b) Liste as idades das pessoas cadastradas (sem repetições)

In [22]:
SELECT DISTINCT Age FROM Pessoa;

c) Liste código das pessoas, idade, código dos medicamentos que usaram e dias de uso

In [23]:
SELECT P.Id, P.Age, U.drug_code, U.days_use
    FROM Pessoa P, Use U
    WHERE P.Id = U.person_id;

d) Liste código das Pessoas, idade, nome dos medicamentos que usaram e quantos dias de uso

In [24]:
SELECT P.Id, P.Age, D.Name, U.days_use
    FROM Pessoa P, Drug D, Use U
    WHERE P.Id = U.person_id AND D.Code = U.drug_code

e) Liste nome dos medicamentos que foram usados por pessoas (sem repetições)

In [25]:
SELECT DISTINCT Name FROM Drug, Use WHERE Drug.Code= Use.drug_code

f) Liste nome dos medicamentos usados por mais de 2.000 dias (sem repetições)

In [26]:
SELECT DISTINCT Name From Drug D, Use U 
    WHERE D.Code = U.drug_code AND U.days_use>2000